导入库与函数

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import geopandas as gpd

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#导入数据并检查字段是否齐全
def import_and_check_data(path, num_fields):
    """
    导入数据并检查字段是否齐全
    :param path: 数据路径
    :param num_fields: 数据字段数量
    :return data: DataFrame
    """
    data = pd.read_csv(path, low_memory=False)
    print("Data Shape:", data.shape)
    print("Data Columns:", data.columns)
    
    if len(data.columns) == num_fields:
        print("数据字段齐全")
    else:
        print("数据字段缺失")
    
    return data

#定义一个查看车辆属性的函数
def extract_vehicle_attributes(data: pd.DataFrame, attributes: list):
    """
    查看车辆属性
    :param data: DataFrame
    :param attributes: 车辆属性列表
    :return results: 车辆属性字典
    """
    
    results = {}
    for attribute in attributes:
        if attribute in data.columns:
            unique_values = data[attribute].unique()
            if len(unique_values) > 1:
                print(f"Attribute {attribute} is not unique.")
            elif pd.isnull(unique_values[0]):
                print(f"Attribute {attribute} is missing.")
            else:
                results[attribute] = unique_values[0]
        else:
            print(f"Attribute {attribute} does not exist in the data.")
    return results

#定义一个批量删除列的函数
def drop_columns(data: pd.DataFrame, columns_to_drop: list):
    data = data.drop(columns=columns_to_drop, errors='ignore')
    print("已删除列名为 {} 的列".format(columns_to_drop))
    return data


# 定义一个删除内部数据值完全相同的列的函数
def remove_duplicate_columns(data):
    """
    删除内部数据值完全相同的列
    :param data: DataFrame
    :return cleaned_data, result_df
    """

    same_col = []
    same_col_dict = []

    for col in data.columns:
        if len(data[col].unique()) == 1:
            same_col.append(col)
            same_col_dict.append(data[col].unique())
        else:
            pass
    
    cleaned_data = data.drop(same_col, axis=1)

    result_df = pd.DataFrame({'same_col': same_col, 'same_col_dict': same_col_dict})
    return cleaned_data, result_df

#定义一个删除重复数据的函数
def drop_duplicated(data):
    print('删除重复数据前：',data.shape)
    print('删除重复的行数为：',data.duplicated().sum())
    data.drop_duplicates(inplace=True)
    print('删除重复数据后：',data.shape)
    return data

#编写一个预处理时间的函数
def time_preprocessing(df,time_col='yr_modahrmn',format='%Y-%m-%d %H:%M:%S'):
    '''
    df:需要处理的数据集
    time_col:时间列的列名
    format:时间格式
    return:处理好的数据集
    '''
    df['time'] = pd.to_datetime(df[time_col],format = format)
    #删除原来的时间列
    df.drop(time_col,axis=1,inplace=True)
    #将时间列设置为索引
    df.set_index('time',inplace=True)
    #按照时间排序
    df.sort_index(inplace=True)
    return df

#编写一个分析特征的函数
def analyze_column(data, column_name,lower_bound = None,upper_bound = None,ratio = None):
    """
    分析特征
    :param data: DataFrame
    :param column_name: 特征名称
    :param upper_bound: 特征的上界
    :param lower_bound: 特征的下界
    :param ratio: 特征放缩比例
    :return:
    """
    # 查看列的数据类型
    print(f"数据类型为 {data[column_name].dtype}")
    

    # 如果数据类型为数字，查看列的统计信息
    if data[column_name].dtype in ['int64', 'float64', 'float32', 'int32']:
        print('-' * 50)
        print(data[column_name].describe())
        print('-' * 50)
        # 画出列的直方图
        data[column_name].hist(bins=50)
        plt.xlabel(column_name)
        plt.ylabel('Frequency')
        plt.title(f'Distribution of {column_name}')
        plt.show()
        print('-' * 50)
        # 画出列随时间的变化图
        plt.figure(figsize=(40,5))
        plt.plot(data.index, data[column_name])
        plt.xlabel('Time')
        plt.ylabel(column_name)
        plt.title(f'{column_name} over Time')
        plt.show()

    # 分析缺失值占比
    missing_number = data[column_name].isnull().sum()
    missing_ratio = missing_number / len(data) * 100

    #打印分割线
    print('-' * 50)
    print(f"'{column_name}' 有 {missing_number} 个缺失值")
    print(f"'{column_name}' 的缺失值占比为 {missing_ratio:.2f}%")

    # 查看列是否在有效值范围
    if upper_bound is not None and lower_bound is not None:
            #打印分割线
        print('-' * 50)
        print(f"有效值范围为 [{lower_bound}, {upper_bound}]")
        invalid_values = data[(data[column_name] < lower_bound) | (data[column_name] > upper_bound)]
        print(f"'{column_name}' 共有 {len(invalid_values)} 个范围外值")
    
    # 是否需要放缩
    if ratio is not None:
        print('-' * 50)
        print(f"需要放缩，放缩比例为 {ratio}")
        data[column_name] = data[column_name].astype('float64')
        data[column_name] = data[column_name] * ratio
        print(f"列放缩完毕，数据类型为 {data[column_name].dtype}")
    

字段信息
| 数据字段名称      | 字段定义         | 字段数据值           |
|-----------------|----------------|--------------------|
| vid             | 车辆ID          | XX_xxx             |
| yr_modahrmn     | 数据采集时间      | yymmdd-hhmmss      |
| vehicle_state   | 车辆状态         | 0x01：车辆启动状态；0x02：熄火；0x03：其他状态；“0xFE”表示异常；“0xFF”表示无效 |
| charging_status | 充电状态         | 0x01：停车充电；0x02：行驶充电；0x03：未充电状态；0x04：充电完成；“0xFE”表示异常；“0xFF”表示无效 |
| mode            | 运行模式         | 0x01: 纯电； 0x02：混动； 0x03：燃油； 0xFE 表示异常； 0xFF 表示无效 |
| speed           | 车速            | 有效值范围：0～2200（表示 0 km/h～220 km/h），最小计量单元：0.1km/h，“0xFF,0xFE”表示异常，“0xFF,0xFF”表示无效 |
| gear            | 挡位            | 挡位定义见表 A.1    |
| total_volt      | 总电压           | 有效值范围：0～10000（表示 0V～1000V），最小计量单元：0.1V，“0xFF,0xFE”表示异常，“0xFF,0xFF”表示无效 |
| total_current   | 总电流           | 有效值范围：0～20000（偏移量 1000A，表示-1000A～+1000A），最小计量单元：0.1A，“0xFF,0xFE”表示异常，“0xFF,0xFF”表示无效 |
| mileage         | 累计里程          | 有效值范围：0～9999999（表示 0km～999999.9km），最小计量单元：0.1km。“0xFF, 0xFF, 0xFF,0xFE”表示异常，“0xFF,0xFF,0xFF,0xFF”表示无效 |
| standard_soc    | SOC-国标         | 有效值范围：0～100（表示 0%～100%），最小计量单元：1%，“0xFE”表示异常，“0xFF”表示无效 |
| cell_volt_list  | 单体电池电压值列表  | 有效值：0~15000，单位0.001V |
| max_volt_num    | 最高电压电池子系统号 | 有效值范围：1～250，“0xFE”表示异常，“0xFF”表示无效 |
| max_cell_volt   | 电池单体电压最高值  | 有效值范围：0～15 000（表示 0V～15V），最小计量单元：0.001V，“0xFE，0xFE”表示异常，“0xFF ，0xFF”表示无效 |
| max_volt_cell_id | 最高电压电池       | 有效值范围： 1～250，“0xFE”表示异常，“0xFF”表示无效。 |
| min_volt_num    | 最低电压电池子系统号 | 有效值范围：1～250，“0xFE”表示异常，“0xFF”表示无效 |
| min_cell_volt   | 电池单体电压最低值  | 有效值范围：0～15 000（表示 0V～15V），最小计量单元：0.001V，“0xFF，0xFE”表示异常，“0xFF ，0xFF”表示无效 |
| min_cell_volt_id | 最低电压电池       | 有效值范围： 1～250，“0xFE”表示异常，“0xFF”表示无效。 |
| max_temp_num    | 最高温度子系统号    | 有效值范围：1～250，“0xFE”表示异常，“0xFF”表示无效 |
| max_temp        | 最高温度值         | 有效值范围：0～250（数值偏移量40℃，表示-40℃～210℃），最小计量单元：1℃，“0xFE”表示异常，“0xFF”表示无效 |
| max_temp_probe_id | 最高温度探针      | 有效值范围： 1～250，“0xFE”表示异常，“0xFF”表示无效。 |
| min_temp_num    | 最低温度子系统号    | 有效值范围：1～250，“0xFE”表示异常，“0xFF”表示无效 |
| min_temp        | 最低温度值         | 有效值范围：0～250（数值偏移量40℃，表示-40℃～210℃），最小计量单元：1℃，“0xFE”表示异常，“0xFF”表示无效 |
| min_temp_probe_id | 最低温度探针      | 有效值范围： 1～250，“0xFE”表示异常，“0xFF”表示无效。 |
| sing_temp_num   | 单体电池温度探针总数 | N个温度探针，有效值范围：1～65531，“0xFF,0xFE”表示异常，“0xFF,0xFF”表示无效 |
| cell_temp_list  | 单体电池温度值列表  | 有效值范围：0～250 （数值偏移量 40℃，表示-40℃～+210℃），最小计量单元：1℃，“0xFE”表示异常，“0xFF”表示无效 |
| max_alarm_lvl   | 最高报警等级      | 为当前发生的故障中的最高等级值，有限制范围0~3，“0”表示无故障；“1”表示1级故障，指不影响车辆正常行驶的故障；“2”表示2级故障，指影响车辆性能，需要驾驶员限制行驶的故障；“3”表示3级故障，为最高级别故障，指驾驶员应立即停车处理或请求救援的故障。具体等级对应的故障内容由厂商自行定义。“0xFE”表示异常，“0xFF”表示无效 |
| bat_fault_list  | 充电储能装置故障代码列表 | 扩展性数据，由厂商自行定义，可充电储能装置故障个数等于可充电储能装置故障总数N1。 |
| isulate_r       | 绝缘阻值         | 有效范围 0～60000（表示 0KΩ ～60000KΩ），最小计量单元：1KΩ |
| dcdc_stat       | DC-DC状态       | 0x01： 工作； 0x02： 断开，“0xFE”表示异常，“0xFF”表示无效。 |
| sing_volt_num   | 单体电池总数      | N个电池单体，有效值范围：1～65531，“0xFF,0xFE”表示异常，“0xFF,0xFF”表示无效 |
| alarm_info      | 通用报警标志      | 通用报警标志位定义见表2 |


In [ ]:
#导入数据
path = '/Users/wanghao/jupyter/data/2022_ncbdc/LB_04/part-00000-d927b640-ae1a-4cad-a86b-77d14c14c00c-c000.csv'
num_fields = 32
data = import_and_check_data(path, num_fields)

In [ ]:
#查看字段种类
data.info()

In [ ]:
# 查看车辆属性值
attributes = ['vid', 'mode', 'sing_temp_num', 'sing_volt_num']
attributes_results = extract_vehicle_attributes(data, attributes)
print(attributes_results)

#删除车辆属性值
data = drop_columns(data, attributes)

这是一台纯电动汽车，电池由95个单体电池组成，34个温度探针

In [ ]:
data = drop_duplicated(data)

数据中的重复行有15990行

## 时间处理

In [ ]:
data = time_preprocessing(data,time_col='yr_modahrmn',format='%Y-%m-%d %H:%M:%S')

## 逐字段分析

### speed
车速：有效值范围：0～2200（表示 0 km/h～220 km/h），最小计量单元：0.1km/h，“0xFF,0xFE”表示异常，“0xFF,0xFF”表示无效

In [ ]:
analyze_column(data, 'speed',0, 2200 , 0.1)

### vehicle_state
车辆状态：0x01：车辆启动状态；0x02：熄火；0x03：其他状态；“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'vehicle_state',0,3)

In [ ]:
#缺失值处理
#打印vehicle_state缺失值所在的行
data[data['vehicle_state'].isnull()]

分析：缺失值待处理，并非整行缺失，可以填补，标签类数据，需要检查是否准确

### charging_status
充电状态：0x01：停车充电；0x02：行驶充电；0x03：未充电状态；0x04：充电完成；“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'charging_status',0,4)

分析：标签类数据，需要检查是否准确

### total_volt	
总电压：有效值范围： 0～10000（表示 0V～1000V），最小计量单元： 0.1V，“0xFF,0xFE”表示异常，“0xFF,0xFF”表示无效

In [ ]:
analyze_column(data, 'total_volt',0, 10000 , 0.1)

分析: 有异常值，待处理

### total_current
有效值范围： 0～20000（偏移量 1000A，表示-1000A～+1000A），最小计量单元： 0.1A，“0xFF,0xFE”表示异常，“0xFF,0xFF”表示无效。

In [ ]:
analyze_column(data, 'total_current',-10000, 10000 , 0.1)

### mileage
累计里程：有效值范围：0～9999999（表示 0km～999999.9km），最小计量单元：0.1km。“0xFF, 0xFF, 0xFF,0xFE”表示异常，“0xFF,0xFF,0xFF,0xFF”表示无效

In [ ]:
analyze_column(data, 'mileage',0, 9999999 , 0.1)

In [ ]:
#检查mileage是否每个后面的值都大于等于前面的值

def check_mileage_order(data):
    mileage_values = data['mileage'].values
    for i in range(1, len(mileage_values)):
        if mileage_values[i] < mileage_values[i - 1]:
            return False
    return True

# 使用之前导入的data数据进行检查
if 'mileage' in data.columns:
    is_mileage_ordered = check_mileage_order(data)
    if is_mileage_ordered:
        print("mileage字段的值是递增的或相等的。")
    else:
        print("mileage字段的值存在不满足递增条件的情况。")
else:
    print("数据中不存在mileage字段。")


### standard_soc
soc:有效值范围：0～100（表示 0%～100%），最小计量单元：1%，“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'standard_soc',0, 100)

### cell_volt_list 
单体电池电压列表：有效值：0~15000，单位0.001V

In [ ]:
analyze_column(data, 'cell_volt_list')

In [ ]:
def check_systems(data: pd.DataFrame, column: str):
    # 定义一个函数来检查一个字符串中":"的数量
    def count_colon(s):
        return s.count(":")
    
    # 使用apply函数检查整个列
    data['colon_count'] = data[column].apply(count_colon)
    
    # 现在，colon_count列中的每个值都是一个整数，表示对应的cell_volt_list值中":"的数量
    # 使用max函数来检查最大的数量
    max_colon_count = data['colon_count'].max()
    
    # 如果最大数量为1，打印"只有一个系统"，否则打印"有多个系统"
    if max_colon_count == 1:
        print("只有一个系统")
    else:
        print("有多个系统")
    
    data= data.drop('colon_count', axis=1)
    return data

# 使用函数
data = check_systems(data, 'cell_volt_list')

分析：电池只有一个子系统

In [ ]:
def process_volt_string(volt_string):
    volt_string = volt_string.split(":")[1]  # 去掉前面的"1:"
    volt_list = volt_string.split("_")  # 分割字符串
    volt_list = [int(volt) for volt in volt_list]  # 转换为整数列表
    volt_list = [volt * 0.001 for volt in volt_list]  # 转换为电压值
    return volt_list

# 然后使用apply函数处理整个列
data['cell_volt_list'] = data['cell_volt_list'].apply(process_volt_string)

In [ ]:
# 计算每个列表的长度
data['volt_list_length'] = data['cell_volt_list'].apply(len)

# 检查是否所有的长度
all_have_volt = all(data['volt_list_length'] == attributes_results["sing_volt_num"])
if all_have_volt:
    print("所有记录都有",attributes_results["sing_volt_num"],"个电压值")
else:
    print("有记录没有",attributes_results["sing_volt_num"],"个电压值")
data= data.drop('volt_list_length', axis=1)

In [ ]:
def find_and_print_out_of_range_rows(data, column_name,lower_bound, upper_bound):
    """
    找出包含不在范围内的数据值的行，并打印出这些行及行数。

    参数:
        data (DataFrame): 包含电压值数据的DataFrame。

    返回:
        None
    """
    # 找出包含不在范围内的数据值的行
    out_of_range_rows = data[data[column_name].apply(lambda volt_list: any(volt < lower_bound or volt > upper_bound for volt in volt_list))]

    # 获取行数
    num_rows = out_of_range_rows.shape[0]

    # 打印行数
    if num_rows == 0:
        print("没有异常或无效值")
    else:
        print(f"异常或无效行数有: {num_rows} 行")
        # 打印出这些行
        print(out_of_range_rows)

In [ ]:
find_and_print_out_of_range_rows(data, 'cell_volt_list',0, 15)

#### max_cell_volt:衍生特征
电池单体电压最高值：有效值范围：0～15 000（表示 0V～15V），最小计量单元：0.001V，“0xFE，0xFE”表示异常，“0xFF ，0xFF”表示无效

In [ ]:
analyze_column(data, 'max_cell_volt',0, 15000,0.001)

#### max_volt_cell_id衍生特征
最高电压电池：有效值范围： 1～250，“0xFE”表示异常，“0xFF”表示无效。


In [ ]:
analyze_column(data, 'max_volt_cell_id',1, 250)

#### min_cell_volt:衍生特征
有效值范围：0～15 000（表示 0V～15V），最小计量单元：0.001V，“0xFF，0xFE”表示异常，“0xFF ，0xFF”表示无效

In [ ]:
analyze_column(data, 'min_cell_volt',0, 15000,0.001)

#### min_cell_volt_id衍生特征
最低电压电池：有效值范围： 1～250，“0xFE”表示异常，“0xFF”表示无效。

In [ ]:
analyze_column(data, 'min_cell_volt_id',1, 250)

In [ ]:
def find_extreme_and_index(lst, mode='max'):
    if mode == 'max':
        value = max(lst)
    elif mode == 'min':
        value = min(lst)
    else:
        raise ValueError("Mode must be 'max' or 'min'")
    index = lst.index(value)+1
    return value, index

def check_volt_consistency(data: pd.DataFrame):
    data['max_volt_and_index'] = data['cell_volt_list'].apply(find_extreme_and_index, mode='max')
    data['min_volt_and_index'] = data['cell_volt_list'].apply(find_extreme_and_index, mode='min')
    
    data['max_cell_volt_real'] = data['max_volt_and_index'].apply(lambda x: x[0])
    data['max_volt_cell_id_real'] = data['max_volt_and_index'].apply(lambda x: x[1])
    
    data['min_cell_volt_real'] = data['min_volt_and_index'].apply(lambda x: x[0])
    data['min_cell_volt_id_real'] = data['min_volt_and_index'].apply(lambda x: x[1])
    
    max_volt_not_equal = (data['max_cell_volt_real'] != data['max_cell_volt'])
    max_volt_not_equal_count = max_volt_not_equal.sum()
    
    min_volt_not_equal = (data['min_cell_volt_real'] != data['min_cell_volt'])
    min_volt_not_equal_count = min_volt_not_equal.sum()
    
    max_id_not_equal = (data['max_volt_cell_id_real'] != data['max_volt_cell_id'])
    max_id_not_equal_count = max_id_not_equal.sum()

    min_id_not_equal = (data['min_cell_volt_id_real'] != data['min_cell_volt_id'])
    min_id_not_equal_count = min_id_not_equal.sum()

    if max_volt_not_equal_count > 0 or max_id_not_equal_count > 0 or min_volt_not_equal_count > 0 or min_id_not_equal_count > 0:
        print(f"警告：存在不相等的情况。")
        print(f"不相等的最大电压值数量：{max_volt_not_equal_count}")
        print(f"不相等的最小电压值数量：{min_volt_not_equal_count}")
        print(f"不相等的最大电压单元ID数量：{max_id_not_equal_count}")
        print(f"不相等的最小电压单元ID数量：{min_id_not_equal_count}")
        return data
    else:
        print("单体电压数据一致性检查完毕，数据一致！")
        # 删除这个函数里新创造的这些列
        data = data.drop(['max_volt_and_index', 'max_cell_volt_real', 'max_volt_cell_id_real', 'min_volt_and_index', 'min_cell_volt_real', 'min_cell_volt_id_real'], axis=1)
        return data


In [ ]:
data = check_volt_consistency(data)

### cell_temp_list
单体电池温度值列表:有效值范围：0～250 （数值偏移量 40℃，表示-40℃～+210℃），最小计量单元：1℃，“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'cell_temp_list')

In [ ]:
def process_temp_string(temp_string):
    temp_list = temp_string.split("_")  # 分割字符串
    temp_list = [int(temp) for temp in temp_list]  # 转换为整数列表
    return temp_list

# 然后使用apply函数处理整个列
data['cell_temp_list'] = data['cell_temp_list'].apply(process_temp_string)

In [ ]:
# 计算每个列表的长度
data['temp_list_length'] = data['cell_temp_list'].apply(len)

# 检查是否所有的长度
all_have_temp = all(data['temp_list_length'] == attributes_results["sing_temp_num"])
if all_have_temp:
    print("所有记录都有",attributes_results["sing_temp_num"],"个温度值")
else:
    print("有记录没有",attributes_results["sing_temp_num"],"个温度值")
data= data.drop('temp_list_length', axis=1)

In [ ]:
find_and_print_out_of_range_rows(data, 'cell_temp_list',0, 250)

分析：总共有10行异常数据,等待排查

#### max_temp：衍生特征
最高温度值 有效值范围：0～250（数值偏移量40℃，表示-40℃～210℃），最小计量单元：1℃，“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'max_temp',-40, 210)

In [ ]:
#打印max_temp缺失值所在的行
data[data['max_temp'].isnull()]

分析：有两个异常值，还有一个0值，待排查

#### max_temp_probe_id：衍生特征
最高温度探针
有效值范围： 1～250，“0xFE”表示异常，“0xFF”表示无效。

In [ ]:
analyze_column(data, 'max_temp_probe_id',1, 250)

#### min_temp  ：衍生特征
最低温度值
有效值范围：0～250（数值偏移量40℃，表示-40℃～210℃），最小计量单元：1℃，“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'min_temp',-40, 210)

分析：图像上有明显的很多异常值

#### min_temp_probe_id：衍生特征
有效值范围： 1～250，“0xFE”表示异常，“0xFF”表示无效。

In [ ]:
analyze_column(data, 'min_temp_probe_id',1, 250)

In [ ]:
def check_temp_consistency(data: pd.DataFrame):
    data['max_temp_and_index'] = data['cell_temp_list'].apply(find_extreme_and_index, mode='max')
    data['min_temp_and_index'] = data['cell_temp_list'].apply(find_extreme_and_index, mode='min')
    
    data['max_temp_real'] = data['max_temp_and_index'].apply(lambda x: x[0])
    data['max_temp_probe_id_real'] = data['max_temp_and_index'].apply(lambda x: x[1])
    
    data['min_temp_real'] = data['min_temp_and_index'].apply(lambda x: x[0])
    data['min_temp_probe_id_real'] = data['min_temp_and_index'].apply(lambda x: x[1])
    
    max_temp_not_equal = (data['max_temp_real'] -40 != data['max_temp'])
    max_temp_not_equal_count = max_temp_not_equal.sum()
    
    min_temp_not_equal = (data['min_temp_real'] -40 != data['min_temp'])
    min_temp_not_equal_count = min_temp_not_equal.sum()
    
    max_id_not_equal = (data['max_temp_probe_id_real'] != data['max_temp_probe_id'])
    max_id_not_equal_count = max_id_not_equal.sum()

    min_id_not_equal = (data['min_temp_probe_id_real'] != data['min_temp_probe_id'])
    min_id_not_equal_count = min_id_not_equal.sum()

    if max_temp_not_equal_count > 0 or max_id_not_equal_count > 0 or min_temp_not_equal_count > 0 or min_id_not_equal_count > 0:
        print(f"警告：存在不相等的情况。")
        print(f"不相等的最大温度值数量：{max_temp_not_equal_count}")
        print(f"不相等的最小温度值数量：{min_temp_not_equal_count}")
        print(f"不相等的最高温度探针ID数量：{max_id_not_equal_count}")
        print(f"不相等的最低温度探针ID数量：{min_id_not_equal_count}")
        return data
    else:
        print("单体温度数据一致性检查完毕，数据一致！")
        # 删除这个函数里新创造的这些列
        data = data.drop(['max_temp_and_index', 'max_temp_real', 'max_temp_probe_id_real', 'min_temp_and_index', 'min_temp_real', 'min_temp_probe_id_real'], axis=1)
        return data


In [ ]:
data = check_temp_consistency(data)

分析：温度与探针数据有不一致，待排查！

### max_alarm_lvl
最高报警等级
为当前发生的故障中的最高等级值，有限制范围0~3，“0”表示无故障；“1”表示1级故障，指不影响车辆正常行驶的故障；“2”表示2级故障，指影响车辆性能，需要驾驶员限制行驶的故障；“3”表示3级故障，为最高级别故障，指驾驶员应立即停车处理或请求救援的故障。具体等级对应的故障内容由厂商自行定义。“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'max_alarm_lvl',0,3)

### bat_fault_list 
充电储能装置故障代码列表
扩展性数据，由厂商自行定义， 可充电储能装置故障个数等于可充电储能装置故障总数 N1。

In [ ]:
analyze_column(data, 'bat_fault_list')

分析：完全缺失

### isulate_r
绝缘阻值
有效范围 0～60000（表示 0KΩ ～60000KΩ），最小计量单元： 1KΩ  

In [ ]:
analyze_column(data, 'isulate_r', 0, 60000)

分析：存在异常值！可能很关键？

### dcdc_stat
DC-DC状态
0x01： 工作； 0x02： 断开，“0xFE”表示异常，“0xFF”表示无效。

In [ ]:
analyze_column(data, 'dcdc_stat',0,2)

### gear
挡位 

In [ ]:
analyze_column(data, 'gear',0,1)

In [ ]:
def parse_gear(gear):
    bit5 = (gear & 0b00100000) >> 5
    bit4 = (gear & 0b00010000) >> 4
    bits3210 = gear & 0b00001111

    gear_dict = {
        0b0000: 0,
        0b0001: 1,
        0b0010: 2,
        0b0011: 3,
        0b0100: 4,
        0b0101: 5,
        0b0110: 6,
        0b1101: 7,
        0b1110: 8,
        0b1111: 9,
    }
    gear_value = gear_dict.get(bits3210, np.nan)  # 如果为未知挡位，输出null
    return bit5, bit4, gear_value


In [ ]:
#档位处理成3列
data['驱动力'], data['制动力'], data['挡位'] = zip(*data['gear'].apply(parse_gear))

In [ ]:
analyze_column(data, '驱动力',0,1)

In [ ]:
analyze_column(data, '制动力',0,1)

In [ ]:
analyze_column(data, '挡位')

分析：这是一台自动档的车，或许可以考虑档位与车速的关系？辅助后续筛查

### 

### 子系统分析：可以忽略

#### max_volt_num
最高电压电池子系统号
有效值范围：1～250，“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'max_volt_num',1,250)

#### min_volt_num
最低电压电池子系统号
有效值范围：1～250，“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'min_volt_num',1,250)

#### max_temp_num
最高温度子系统号
有效值范围：1～250，“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'max_temp_num',1,250)

#### min_temp_num
最低温度子系统号
有效值范围：1～250，“0xFE”表示异常，“0xFF”表示无效

In [ ]:
analyze_column(data, 'min_temp_num',1,250)

分析：都只有一个子系统，所以不需要进行分析

### alarm_info
通用报警标志

In [ ]:
analyze_column(data, 'alarm_info')

报警位说明
| 位 | 定义 | 处理说明 |
|---|---|---|
| 0 | 1：温度差异报警；0：正常 | 标志维持到报警条件解除 |
| 1 | 1：电池高温报警；0：正常 | 标志维持到报警条件解除 |
| 2 | 1：车载储能装置类型过压报警；0：正常 | 标志维持到报警条件解除 |
| 3 | 1：车载储能装置类型欠压报警；0：正常 | 标志维持到报警条件解除 |
| 4 | 1：SOC低报警；0：正常 | 标志维持到报警条件解除 |
| 5 | 1：单体电池过压报警；0：正常 | 标志维持到报警条件解除 |
| 6 | 1：单体电池欠压报警；0：正常 | 标志维持到报警条件解除 |
| 7 | 1：SOC过高报警；0：正常 | 标志维持到报警条件解除 |
| 8 | 1：SOC跳变报警；0：正常 | 标志维持到报警条件解除 |
| 9 | 1：可充电储能系统不匹配报警；0：正常 | 标志维持到报警条件解除 |
| 10 | 1：电池单体一致性差报警；0：正常 | 标志维持到报警条件解除 |
| 11 | 1：绝缘报警；0：正常 | 标志维持到报警条件解除 |
| 12 | 1：DC-DC温度报警；0：正常 | 标志维持到报警条件解除 |
| 13 | 1：制动系统报警；0：正常 | 标志维持到报警条件解除 |
| 14 | 1：DC-DC状态报警；0：正常 | 标志维持到报警条件解除 |
| 15 | 1：驱动电机控制器温度报警；0：正常 | 标志维持到报警条件解除 |
| 16 | 1：高压互锁状态报警；0：正常 | 标志维持到报警条件解除 |
| 17 | 1：驱动电机温度报警；0：正常 | 标志维持到报警条件解除 |
| 18 | 1：车载储能装置类型过充；0：正常 | 标志维持到报警条件解除 |
| 19~31 | 预留 | 标志维持到报警条件解除 |


分析：该车出现了 DC_DC状态的报警，需要和前面的max_alarm_lvl进行一致性判断